# HOMEWORK UNIT 1 - Violeta Júlvez Ibáñez

In this project, we will work with the following continuous probability distributions:

1. **Normal (Gaussian) distribution $\mathcal{N}(\mu, \sigma^2)$**  
   We analyze different combinations of mean $\mu$ and variance $\sigma^2$:
   - $\mathcal{N}(0, 1)$  
   - $\mathcal{N}(1, 1)$  
   - $\mathcal{N}(0, 3)$

2. **Uniform distribution**  
   The continuous uniform distribution on the interval $[0,1]$:
   - $\text{Unif}(0,1)$

3. **Beta distribution**  
   Beta distributions with different shape parameters:
   - $\text{Beta}(\alpha = 2, \beta = 5)$  
   - $\text{Beta}(\alpha = 5, \beta = 2)$



We will use **Bokeh** as our visualization library, since it provides interactive, high-resolution plots that make it easy to visually compare different distributions and confidence intervals. This library is specially good for exploratory data analysis.

In [2]:
!pip install bokeh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 10.4 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 14.4 MB/s  0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9/9 [bokeh]32m8/9 [bokeh]]s]


We first generate the samples for the different probability distributions:

In [41]:
import numpy as np
import pandas as pd
from scipy.stats import norm, beta, uniform

from bokeh.io import output_notebook, show
from bokeh.plotting import figure

# Enable Bokeh plotting inside the notebook
output_notebook()

np.random.seed(127)
n = 1000

# Normal distribution 
normal_params = [
    (0.0, 1.0),
    (1.0, 1.0),
    (0.0, 3.0)
]
normal_samples = [
    np.random.normal(loc=mu, scale=np.sqrt(var), size=n)
    for (mu, var) in normal_params
]

# Uniform(0,1)
uniform_samples = np.random.uniform(0, 1, size=n)

# Beta distribution
beta_params = [
    (2.0, 5.0),
    (5.0, 2.0)
]
beta_samples = [
    np.random.beta(a, b, size=n)
    for (a, b) in beta_params
]


Loading BokehJS ...

In [77]:
from bokeh.layouts import gridplot
from bokeh.palettes import Category10

# Function to create the visualizations of the different distributins (histogram + density curve)
def bokeh_hist_with_pdf_colored(samples, pdf_func=None, pdf_label="pdf",
                                title="Histogram", x_range=None, bins=40,
                                hist_color="#1f77b4", pdf_color="#1f77b4"):
    # Normalized histogram
    hist, edges = np.histogram(samples, density=True, bins=bins)

    p = figure(title=title, width=400, height=300, x_range=x_range)

    # Histogram bars
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
           fill_alpha=0.35, line_alpha=0.8,
           fill_color=hist_color, line_color=hist_color,
           legend_label="Histogram")

    # Theoretical density curve
    if pdf_func is not None:
        xs = np.linspace(edges[0], edges[-1], 300)
        ys = pdf_func(xs)
        p.line(xs, ys, line_width=2, legend_label=pdf_label,
               line_color=pdf_color)

    p.legend.location = "top_left"
    p.xaxis.axis_label = "x"
    p.yaxis.axis_label = "Density"
    return p

In [90]:
# NORMAL DISTRIBUTIONS

x_min, x_max = -6, 6
normal_plots = []

for (color, ((mu, var), samp)) in zip(Category10[3], zip(normal_params, normal_samples)):
    sigma = np.sqrt(var)
    title = f"Normal(mu={mu}, sigma^2={var})"
    pdf_func = lambda x, m=mu, s=sigma: norm.pdf(x, loc=m, scale=s)

    p_n = bokeh_hist_with_pdf_colored(
        samp,
        pdf_func=pdf_func,
        pdf_label="Normal pdf",
        title=title,
        x_range=(x_min, x_max),
        hist_color=color,
        pdf_color=color
    )
    normal_plots.append(p_n)

show(gridplot([normal_plots], toolbar_location="right"))

In [28]:
# UNIFORM DISTRIBUTION

uniform_plot = bokeh_hist_with_pdf_colored(
    uniform_samples,
    pdf_func=lambda x: uniform.pdf(x, loc=0, scale=1),
    pdf_label="Uniform(0,1) pdf",
    title="Uniform(0,1)",
    x_range=(0, 1),
    bins=30,
    hist_color=Category10[3][1],
    pdf_color=Category10[3][1]
)

show(gridplot([[uniform_plot]], toolbar_location="right"))

In [32]:
# BETA DISTRIBUTION

beta_plots = []

for (color, ((a_, b_), samp)) in zip(beta_colors, zip(beta_params, beta_samples)):
    title = f"Beta(a={a_}, b={b_})"
    pdf_func = lambda x, aa=a_, bb=b_: beta.pdf(x, aa, bb)
    p = bokeh_hist_with_pdf_colored(
        samp,
        pdf_func=pdf_func,
        pdf_label="Beta pdf",
        title=title,
        x_range=(0, 1),
        bins=30,
    )
    beta_plots.append(p)

# Show betas side by side
show(gridplot([beta_plots], toolbar_location="right"))


#### **PROBLEM 1**
* **Estimate and compare the confidence intervals or error bars obtained for each distribution using Hoeffding's inequality and the Chebyshev inequality (for the latter one, you need to analyze or empirically  estimate the variance).** 

A **confidence interval (CI)** provides a range of plausible values for an unknown parameter of a distribution.  
In this case, the parameter of interest is the **mean** of a random variable $X$:

$$
\mu = \mathbb{E}[X].
$$

Given a sample of observations
$$
X_1, X_2, \dots, X_n,
$$
we compute the **sample mean**:
$$
\bar{X} = \frac{1}{n} \sum_{i=1}^{n} X_i.
$$

A confidence interval for $\theta$ is an interval of the form

$$
\big[\, \bar{X} - \varepsilon,\; \bar{X} + \varepsilon \,\big],
$$

where $\varepsilon > 0$ is chosen such that:

$$
\mathbb{P} \left(\, \mu \in [\bar{X} - \varepsilon,\; \bar{X} + \varepsilon] \,\right)
\ge 1 - \delta.
$$

Where:

- $1 - \delta$ is the **confidence level**
- $\varepsilon$ determines the **width** of the interval
- The inequality above ensures that the interval contains the true mean $\mu$ with probability at least $1 - \delta$


For this problem we are going to choose a confidence level $1-\delta =95\%$.

1. Let us first compute the confidence intervals for the mean using the **Chebychev's** inequality:

Chebyshev's inequality says:
$$
\mathbb{P}\big( |\bar{X} - \mu| \ge \varepsilon \big)
\le
\frac{\mathrm{Var}(\bar{X})}{\varepsilon^2}
=
\frac{\sigma^2 / n}{\varepsilon^2}
=
\frac{\sigma^2}{n \varepsilon^2}.
$$

Where $\mathrm{Var}(\bar{X}) = \frac{\sigma^2}{n}$ and $\varepsilon$ is any positive number.

We want to construct a **confidence interval** of the form
$$
[\bar{X} - \varepsilon,\; \bar{X} + \varepsilon]
$$
We want the true mean $\mu$ to lie in this interval with probability at least $1 - \delta$.

$$
\mathbb{P}\big( |\bar{X} - \mu| \ge \varepsilon \big) \le \delta.
$$

Therefore, it is enough to choose $\varepsilon$ such that:
$$
\frac{\sigma^2}{n \varepsilon^2} \le \delta.
$$


Thus, a $(1 - \delta)$ **Chebyshev confidence interval** for the mean is
$$
[\bar{X} - \varepsilon_C,\; \bar{X} + \varepsilon_C],
\quad
\text{with }
\varepsilon_C = \sqrt{ \frac{\sigma^2}{n \delta} }.
$$


In practice, the true variance $\sigma^2$ is unknown.  
We replace it by the **empirical variance** (sample variance):
$$
\widehat{\sigma}^2 = \frac{1}{n - 1} \sum_{i=1}^{n} (X_i - \bar{X})^2.
$$

Then we use
$$
\varepsilon_C \approx \sqrt{ \frac{\widehat{\sigma}^2}{n \delta} },
$$

In [51]:
from bokeh.models import ColumnDataSource

# Confidence level: 95%
delta = 0.05
conf_level = 1 - delta

def chebyshev_ci(samples, delta):
    """
    Chebyshev confidence interval for the mean.
    Uses the empirical variance.
    Returns: lower_bound, upper_bound, epsilon, mean_hat, var_hat
    """
    n = len(samples)
    mean_hat = np.mean(samples)
    var_hat = np.var(samples, ddof=1)  # sample variance
    eps = np.sqrt(var_hat / (n * delta))
    return mean_hat - eps, mean_hat + eps, eps, mean_hat, var_hat

rows = []

# Normal distributions
for (mu, var), samp in zip(normal_params, normal_samples):
    lo, hi, eps, mean_hat, var_hat = chebyshev_ci(samp, delta)
    rows.append({
        "Distribution": f"Normal(mu={mu}, var={var})",
        "Empirical mean": mean_hat,
        "Empirical variance": var_hat,
        "epsilon_C": eps,
        f"{int(conf_level*100)}% CI lower": lo,
        f"{int(conf_level*100)}% CI upper": hi,
        "CI width": hi - lo
    })

# Uniform(0,1)
lo_u, hi_u, eps_u, mean_u, var_u = chebyshev_ci(uniform_samples, delta)
rows.append({
    "Distribution": "Uniform(0,1)",
    "Empirical mean": mean_u,
    "Empirical variance": var_u,
    "epsilon_C": eps_u,
    f"{int(conf_level*100)}% CI lower": lo_u,
    f"{int(conf_level*100)}% CI upper": hi_u,
    "CI width": hi_u - lo_u
})

# Beta distrbutions
for (a_, b_), samp in zip(beta_params, beta_samples):
    lo_b, hi_b, eps_b, mean_b, var_b = chebyshev_ci(samp, delta)
    rows.append({
        "Distribution": f"Beta(a={a_}, b={b_})",
        "Empirical mean": mean_b,
        "Empirical variance": var_b,
        "epsilon_C": eps_b,
        f"{int(conf_level*100)}% CI lower": lo_b,
        f"{int(conf_level*100)}% CI upper": hi_b,
        "CI width": hi_b - lo_b
    })


The results are:

In [52]:
chebyshev_table = pd.DataFrame(rows)
chebyshev_table

,Distribution,Empirical mean,Empirical variance,epsilon_C,95% CI lower,95% CI upper,CI width
0,"Normal(mu=0.0, var=1.0)",-0.0103,1.0295,0.1435,-0.1538,0.1332,0.2870
1,"Normal(mu=1.0, var=1.0)",0.9661,0.9539,0.1381,0.8280,1.1042,0.2762
2,"Normal(mu=0.0, var=3.0)",0.1314,3.2444,0.2547,-0.1233,0.3861,0.5095
3,"Uniform(0,1)",0.5051,0.0853,0.0413,0.4638,0.5464,0.0826
4,"Beta(a=2.0, b=5.0)",0.2851,0.0249,0.0223,0.2628,0.3075,0.0447
5,"Beta(a=5.0, b=2.0)",0.7248,0.0245,0.0221,0.7027,0.7469,0.0442


And let us plot these results:

In [53]:
# Prepare data for Bokeh from the table
dist_labels = chebyshev_table["Distribution"].tolist()
means = chebyshev_table["Empirical mean"].tolist()
ci_lower = chebyshev_table[f"{int(conf_level*100)}% CI lower"].tolist()
ci_upper = chebyshev_table[f"{int(conf_level*100)}% CI upper"].tolist()

source = ColumnDataSource(data=dict(
    dist=dist_labels,
    mean=means,
    ci_lower=ci_lower,
    ci_upper=ci_upper,
))

p = figure(
    x_range=dist_labels,
    width=900,
    height=400,
    title=f"Chebyshev {int(conf_level*100)}% Confidence Intervals for the Mean",
    toolbar_location="right"
)
p.segment(
    x0='dist', y0='ci_lower',
    x1='dist', y1='ci_upper',
    source=source,
    line_width=3,
    color="orange",
    legend_label="Chebyshev CI"
)
p.circle(
    x='dist', y='mean',
    size=8,
    source=source,
    color="black",
    legend_label="Empirical mean"
)

p.xaxis.axis_label = "Distribution"
p.yaxis.axis_label = "Mean and confidence interval"
p.xaxis.major_label_orientation = 0.7
p.legend.location = "top_left"

show(p)


Following the plot we can see that distributions with larger variance, like the wider Normal, produce much wider Chebyshev intervals. The Uniform(0,1) distribution yields moderate interval due to its relatively small variance.
The Beta distributions give the smallest intervals, as their variance is very small (most of the probability mass lies very close to the mean as we can see in the beta plots).

**In conclusion**: Chebyshev's intervals mainly reflect the variance of each distribution — higher variance → wider interval and lower variance → tighter interval.

2. Now let us compute de mean confidence intervals using **Hoeffding**'s inequality. 

Under the assumption that the random variables are **bounded**.

$$
a \le X_i \le b \quad \text{for all } i.
$$

Hoeffding’s inequality states that for any $\varepsilon > 0$:
$$
\mathbb{P}\left( |\bar{X} - \mathbb{E}[X]| \ge \varepsilon \right)
\le
2 \,\exp\!\left( -\frac{2n\varepsilon^2}{(b - a)^2} \right).
$$

For a confidence level $1 - \delta$, solving for $\varepsilon$ gives:

$$
\mathbb{P}\left( |\bar{X} - \mu| \ge \varepsilon \right) \le \delta.
$$
Ç
$$
2 \exp\!\left( -\frac{2n\varepsilon^2}{(b - a)^2} \right) \le \delta.
$$


$$
\varepsilon_H =
\sqrt{ \frac{(b - a)^2}{2n} \, \ln\left( \frac{2}{\delta} \right) } .
$$

Therefore, a $(1 - \delta)$ Hoeffding confidence interval for the mean is
$$
\big[ \bar{X} - \varepsilon_H,\; \bar{X} + \varepsilon_H \big].
$$

In [63]:
def hoeffding_ci(samples, delta, a, b):
    """
    Hoeffding confidence interval for the mean of bounded variables in [a, b].
    Returns: lower_bound, upper_bound, epsilon, mean_hat
    """
    n = len(samples)
    mean_hat = np.mean(samples)
    eps = np.sqrt(((b - a)**2 / (2 * n)) * np.log(2 / delta))
    return mean_hat - eps, mean_hat + eps, eps, mean_hat

rows = []

# Normal distributions (using empirical min/max as pseudo-bounds)
for (mu, var), samp in zip(normal_params, normal_samples):
    a_n = np.min(samp)   # pseudo lower bound from the sample
    b_n = np.max(samp)   # pseudo upper bound from the sample
    lo_n, hi_n, eps_n, mean_n = hoeffding_ci(samp, delta, a_n, b_n)
    rows.append({
        "Distribution": f"Normal(mu={mu}, var={var})",
        "Empirical mean": mean_n,
        "a (lower bound)": a_n,
        "b (upper bound)": b_n,
        "epsilon_H": eps_n,
        f"{int(conf_level*100)}% CI lower": lo_n,
        f"{int(conf_level*100)}% CI upper": hi_n,
        "CI width": hi_n - lo_n
    })

# Uniform(0,1)
a_u, b_u = 0.0, 1.0
lo_u, hi_u, eps_u, mean_u = hoeffding_ci(uniform_samples, delta, a_u, b_u)
rows.append({
    "Distribution": "Uniform(0,1)",
    "Empirical mean": mean_u,
    "a (lower bound)": a_u,
    "b (upper bound)": b_u,
    "epsilon_H": eps_u,
    f"{int(conf_level*100)}% CI lower": lo_u,
    f"{int(conf_level*100)}% CI upper": hi_u,
    "CI width": hi_u - lo_u
})

# Beta distributions
for (a_param, b_param), samp in zip(beta_params, beta_samples):
    a_b, b_b = 0.0, 1.0
    lo_b, hi_b, eps_b, mean_b = hoeffding_ci(samp, delta, a_b, b_b)
    rows.append({
        "Distribution": f"Beta(a={a_param}, b={b_param})",
        "Empirical mean": mean_b,
        "a (lower bound)": a_b,
        "b (upper bound)": b_b,
        "epsilon_H": eps_b,
        f"{int(conf_level*100)}% CI lower": lo_b,
        f"{int(conf_level*100)}% CI upper": hi_b,
        "CI width": hi_b - lo_b
    })

hoeffding_table = pd.DataFrame(rows)
hoeffding_table


,Distribution,Empirical mean,a (lower bound),b (upper bound),epsilon_H,95% CI lower,95% CI upper,CI width
0,"Normal(mu=0.0, var=1.0)",-0.0103,-3.1687,3.0837,0.2685,-0.2788,0.2582,0.5370
1,"Normal(mu=1.0, var=1.0)",0.9661,-2.1740,4.1758,0.2727,0.6934,1.2388,0.5454
2,"Normal(mu=0.0, var=3.0)",0.1314,-5.7171,6.1012,0.5076,-0.3762,0.6389,1.0151
3,"Uniform(0,1)",0.5051,0.0000,1.0000,0.0429,0.4622,0.5481,0.0859
4,"Beta(a=2.0, b=5.0)",0.2851,0.0000,1.0000,0.0429,0.2422,0.3281,0.0859
5,"Beta(a=5.0, b=2.0)",0.7248,0.0000,1.0000,0.0429,0.6819,0.7678,0.0859


We plot the intervals:

In [65]:
# Prepare data for Bokeh from hoeffding_table
dist_labels2 = hoeffding_table["Distribution"].tolist()
means2 = hoeffding_table["Empirical mean"].tolist()
ci_lower2 = hoeffding_table[f"{int(conf_level*100)}% CI lower"].tolist()
ci_upper2 = hoeffding_table[f"{int(conf_level*100)}% CI upper"].tolist()

source2 = ColumnDataSource(data=dict(
    dist=dist_labels2,
    mean=means2,
    ci_lower=ci_lower2,
    ci_upper=ci_upper2,
))

p2 = figure(
    x_range=dist_labels2,
    width=800,
    height=400,
    title=f"Hoeffding {int(conf_level*100)}% Confidence Intervals for the Mean",
    toolbar_location="right"
)
p2.segment(
    x0='dist', y0='ci_lower',
    x1='dist', y1='ci_upper',
    source=source2,
    line_width=3,
    color="navy",
    legend_label="Hoeffding CI"
)
p2.circle(
    x='dist', y='mean',
    size=8,
    source=source2,
    color="black",
    legend_label="Empirical mean"
)

p2.xaxis.axis_label = "Distribution"
p2.yaxis.axis_label = "Mean and confidence interval"
p2.xaxis.major_label_orientation = 0.7
p2.legend.location = "top_left"

show(p2)


**Conclusion**: From the table and the plot we see that, for the bounded distributions (Uniform(0,1) and the Beta distributions), Hoeffding’s inequality produces **exactly the same confidence interval width**, since all of them share the same support $[0,1]$. The bound depends only on the range $(b - a)$ and the sample size $n$, not on the variance or the specific shape of the distribution. Unlike Chebyshev’s inequality, no variance estimation is required.

For the Normal distributions, whose support is mathematically unbounded $(\mathbb{R})$, Hoeffding’s inequality does **not** theoretically apply. To illustrate its behaviour, we used the empirical minimum and maximum values from the sample as **pseudo-bounds**. Therefore, the resulting intervals for the Normal case should be interpreted only as approximations and **not** as valid Hoeffding confidence intervals. In this case the resulting intervals are **not equal** because the supports are different since each Normal distribution produces different minimum and maximum observed value.


Let us now compare the Hoeffding's intervals with Chebyshev's:

In [70]:
# Merge cleanly and inspect columns
merged = hoeffding_table.merge(
    chebyshev_table,
    on="Distribution",
    suffixes=("_H", "_C")
)

dist_all = merged["Distribution"].tolist()
means_all = merged["Empirical mean_H"].tolist()   # same for _C

hoe_lo_all = merged["95% CI lower_H"].tolist()
hoe_hi_all = merged["95% CI upper_H"].tolist()

cheb_lo_all = merged["95% CI lower_C"].tolist()
cheb_hi_all = merged["95% CI upper_C"].tolist()

merged

,Distribution,Empirical mean_H,a (lower bound),b (upper bound),epsilon_H,95% CI lower_H,95% CI upper_H,CI width_H,Empirical mean_C,Empirical variance,epsilon_C,95% CI lower_C,95% CI upper_C,CI width_C
0,"Normal(mu=0.0, var=1.0)",-0.0103,-3.1687,3.0837,0.2685,-0.2788,0.2582,0.5370,-0.0103,1.0295,0.1435,-0.1538,0.1332,0.2870
1,"Normal(mu=1.0, var=1.0)",0.9661,-2.1740,4.1758,0.2727,0.6934,1.2388,0.5454,0.9661,0.9539,0.1381,0.8280,1.1042,0.2762
2,"Normal(mu=0.0, var=3.0)",0.1314,-5.7171,6.1012,0.5076,-0.3762,0.6389,1.0151,0.1314,3.2444,0.2547,-0.1233,0.3861,0.5095
3,"Uniform(0,1)",0.5051,0.0000,1.0000,0.0429,0.4622,0.5481,0.0859,0.5051,0.0853,0.0413,0.4638,0.5464,0.0826
4,"Beta(a=2.0, b=5.0)",0.2851,0.0000,1.0000,0.0429,0.2422,0.3281,0.0859,0.2851,0.0249,0.0223,0.2628,0.3075,0.0447
5,"Beta(a=5.0, b=2.0)",0.7248,0.0000,1.0000,0.0429,0.6819,0.7678,0.0859,0.7248,0.0245,0.0221,0.7027,0.7469,0.0442


In [94]:
source_all = ColumnDataSource(data=dict(
    dist=dist_all,
    mean=means_all,
    hoe_lo=hoe_lo_all,
    hoe_hi=hoe_hi_all,
    cheb_lo=cheb_lo_all,
    cheb_hi=cheb_hi_all,
))

p_all = figure(
    x_range=dist_all,
    width=1000,
    height=450,
    title="Hoeffding vs Chebyshev (correct CI extraction)",
    toolbar_location="right"
)



# Hoeffding (navy)
p_all.segment(
    x0="dist", y0="hoe_lo",
    x1="dist", y1="hoe_hi",
    source=source_all,
    line_width=4,
    color="navy",
    legend_label="Hoeffding CI"
)
# Chebyshev (orange)
p_all.segment(
    x0="dist", y0="cheb_lo",
    x1="dist", y1="cheb_hi",
    source=source_all,
    line_width=4,
    color="orange",
    legend_label="Chebyshev CI"
)

# Means
p_all.circle(
    x="dist", y="mean",
    size=8,
    source=source_all,
    color="black",
    legend_label="Empirical mean"
)

p_all.xaxis.major_label_orientation = 0.8
p_all.legend.location = "top_left"

show(p_all)


**Conclusion**:

In this cse, Chebyshev’s intervals are actually shorter than Hoeffding’s for the Uniform and Beta distributions. This is because Chebyshev exploits the small empirical variance of these distributions, while Hoeffding only uses the worst-case range $(b - a) = 1$, leading to a more conservative bound.
For the Normal distributions, Chebyshev also produces narrower intervals than Hoeffding. In this case Hoeffding relies on empirical min/max pseudo-bounds, and since the Gaussian support is $\mathbb{R}$ these bounds are not theoretically justified; the resulting intervals are only approximate.


#### **PROBLEM 2**
**When possible, compare your results to theoretical values.**

Our empirical results are actually close to the theoretical values as a direct consequence of classical limit theorems in probability and statistics.

1. Sample mean and variance

From a sample $X_1, \dots, X_n$ we compute:

- the sample mean
  $$
  \hat{\mu} = \bar{X} = \frac{1}{n} \sum_{i=1}^n X_i,
  $$
- and the sample variance
  $$
  \hat{\sigma}^2 = \frac{1}{n-1} \sum_{i=1}^n (X_i - \hat{\mu})^2.
  $$

By the **Law of Large Numbers (LLN)**, as $n \to \infty$,
$$
\hat{\mu} \to \mu \quad \text{and} \quad \hat{\sigma}^2 \to \sigma^2
$$

This means that for a reasonably large sample size such as $n = 1000$ (like our case), we expect the empirical mean and variance to be very close to their theoretical counterparts.

2. Chebyshev $\varepsilon_C$

Chebyshev’s inequality for the sample mean uses the variance of the underlying distribution:
$$
\varepsilon_C^{\text{(theoretical)}} 
= \sqrt{\frac{\sigma^2}{n \delta}}.
$$

In practice, $\sigma^2$ is unknown and we replace it by the sample variance:
$$
\varepsilon_C^{\text{(empirical)}} 
= \sqrt{\frac{\hat{\sigma}^2}{n \delta}}.
$$

Since $\hat{\sigma}^2$ is a **consistent estimator** of $\sigma^2$ (again by LLN-type arguments and basic asymptotic theory), we have
$$
\hat{\sigma}^2 \approx \sigma^2
\quad \Rightarrow \quad
\varepsilon_C^{\text{(empirical)}} \approx \varepsilon_C^{\text{(theoretical)}}.
$$


3. Hoeffding $\varepsilon_H$

For the bounded distributions (Uniform(0,1) and the Beta distributions), Hoeffding’s radius is
$$
\varepsilon_H^{\text{(theoretical)}} 
= \sqrt{\frac{(b - a)^2}{2n} \ln\left(\frac{2}{\delta}\right)},
$$
with $a = 0$ and $b = 1$.

Importantly, this expression does **not** depend on the data, only on $(b-a)$, $n$ and $\delta$.  
Therefore, the empirical Hoeffding radius we compute in the code coincides exactly with the theoretical one for these distributions.

However, notice that Hoeffding’s inequality is not theoretically aplicable to the Normal distributions because the Gaussian support is unbounded ($\mathbb{R}$). To compute a Hoeffding-type interval we used empirical pseudo-bounds based on the observed minimum and maximum of each sample. These artificial bounds depend heavily on sampling variability and do not reflect any true boundedness property of the Normal distribution. As a consequence, the resulting Hoeffding intervals may be misleading: they may underperform, fail to include the theoretical mean, or give a false impression of precision despite having no theoretical guarantee.

This comparison highlights an important conceptual difference: Chebyshev relies on the empirical variance and remains valid for unbounded distributions, while Hoeffding requires strict boundedness and becomes unreliable when this assumption is violated. Therefore, for the Normal case, only the Chebyshev intervals should be interpreted as meaningful confidence guarantees.

In summary, our empirical results are really close to the theoretical quantities as a direct consequence of the Law of Large Numbers and the consistency of the variance estimator.

#### **PROBLEM 3**
**For which distributions does the 68–95–99.7 rule hold?**

The **68–95–99.7 rule** (also known as the *empirical rule*) states that for a **Normal distribution** with mean $\mu$ and standard deviation $\sigma$:

- About **68%** of the probability mass lies within **one** standard deviation of the mean:
  $$
  \mathbb{P}(|X - \mu| \le \sigma) \approx 0.68.
  $$
- About **95%** lies within **two** standard deviations:
  $$
  \mathbb{P}(|X - \mu| \le 2\sigma) \approx 0.95.
  $$
- About **99.7%** lies within **three** standard deviations:
  $$
  \mathbb{P}(|X - \mu| \le 3\sigma) \approx 0.997.
  $$

In [74]:
def empirical_empirical_rule(samples, name):
    """
    Compute empirical proportions within 1, 2, 3 standard deviations from the sample mean.
    """
    mean_hat = np.mean(samples)
    std_hat = np.std(samples, ddof=1)
    
    props = {}
    for k, label in zip([1, 2, 3], ["1σ", "2σ", "3σ"]):
        inside = np.mean(np.abs(samples - mean_hat) <= k * std_hat)
        props[label] = inside
    return mean_hat, std_hat, props

rows = []

# Normal distributions
for (mu, var), samp in zip(normal_params, normal_samples):
    mean_hat, std_hat, props = empirical_empirical_rule(samp, "normal")
    rows.append({
        "Distribution": f"Normal(mu={mu}, var={var})",
        "Empirical mean": mean_hat,
        "Empirical std": std_hat,
        "P(|X-μ| ≤ 1σ)": props["1σ"],
        "P(|X-μ| ≤ 2σ)": props["2σ"],
        "P(|X-μ| ≤ 3σ)": props["3σ"],
    })

# Uniform(0,1)
mean_hat_u, std_hat_u, props_u = empirical_empirical_rule(uniform_samples, "uniform")
rows.append({
    "Distribution": "Uniform(0,1)",
    "Empirical mean": mean_hat_u,
    "Empirical std": std_hat_u,
    "P(|X-μ| ≤ 1σ)": props_u["1σ"],
    "P(|X-μ| ≤ 2σ)": props_u["2σ"],
    "P(|X-μ| ≤ 3σ)": props_u["3σ"],
})

# Beta distributions
for (a_, b_), samp in zip(beta_params, beta_samples):
    mean_hat_b, std_hat_b, props_b = empirical_empirical_rule(samp, "beta")
    rows.append({
        "Distribution": f"Beta(a={a_}, b={b_})",
        "Empirical mean": mean_hat_b,
        "Empirical std": std_hat_b,
        "P(|X-μ| ≤ 1σ)": props_b["1σ"],
        "P(|X-μ| ≤ 2σ)": props_b["2σ"],
        "P(|X-μ| ≤ 3σ)": props_b["3σ"],
    })

rule_table = pd.DataFrame(rows)
rule_table


,Distribution,Empirical mean,Empirical std,P(|X-μ| ≤ 1σ),P(|X-μ| ≤ 2σ),P(|X-μ| ≤ 3σ)
0,"Normal(mu=0.0, var=1.0)",-0.0103,1.0146,0.6800,0.9590,0.9970
1,"Normal(mu=1.0, var=1.0)",0.9661,0.9767,0.6860,0.9530,0.9980
2,"Normal(mu=0.0, var=3.0)",0.1314,1.8012,0.6710,0.9560,0.9960
3,"Uniform(0,1)",0.5051,0.2921,0.5680,1.0000,1.0000
4,"Beta(a=2.0, b=5.0)",0.2851,0.1579,0.6630,0.9660,0.9970
5,"Beta(a=5.0, b=2.0)",0.7248,0.1564,0.6560,0.9580,0.9970


Based on the empirical proportions within 1, 2 and 3 standard deviations from the sample mean, the 68–95–99.7 rule holds **only for the Normal distributions**. For all three Gaussians, the observed proportions are very close to $(0.68, 0.95, 0.997)$.

For the Uniform(0,1) and Beta distributions, the proportions deviate significantly from these values: the mass is either too evenly spread (Uniform) or too concentrated near the mean (Betas), so the 68–95–99.7 rule does **not** hold for these distributions.

We can see:

In [88]:
from bokeh.models import BoxAnnotation

# Helper to add ±1σ, ±2σ, ±3σ bands to the plts
def add_sigma_bands(p, samples):
    mean_hat = np.mean(samples)
    std_hat = np.std(samples, ddof=1)

    # Colors: 1σ = yellow, 2σ = orange, 3σ = red
    colors = [
        ("1σ", "#fff59d", 0.45),
        ("2σ", "#ffcc80", 0.30),
        ("3σ", "#ef9a9a", 0.20)
    ]

    for k, (label, color, alpha) in zip([1, 2, 3], colors):
        band = BoxAnnotation(
            left=mean_hat - k * std_hat,
            right=mean_hat + k * std_hat,
            fill_alpha=alpha,
            fill_color=color
        )
        p.add_layout(band)

    return mean_hat, std_hat


In [91]:
normal_plots_ = normal_plots
normal_plots_bands = []

for plot in normal_plots:
    add_sigma_bands(plot, normal_samples)
    normal_plots_bands.append(plot)
    
show(gridplot([normal_plots_bands], toolbar_location="right"))

In [92]:
uniform_plot_bands = uniform_plot

add_sigma_bands(uniform_plot, uniform_samples)

show(gridplot([[uniform_plot]], toolbar_location="right"))

In [93]:
beta_plots_ = beta_plots
beta_plots_bands = []

for plot in beta_plots_:
    
    add_sigma_bands(plot, beta_samples)

    beta_plots_bands.append(plot)

show(gridplot([beta_plots_bands], toolbar_location="right"))
